# Title
by: Jessie Liang, Jennifer Tsang

In [1]:
import numpy as np
import pandas as pd
import altair as alt
from datetime import datetime as dt

# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [2]:
# load data
items = pd.read_csv("data/items.csv")
original_sales = pd.read_csv("data/sales.csv")

In [3]:
# view items dataset
items['product_profit'] = items['price'] - items['production_cost']
items

,item_name,price,production_cost,item_type,drink_temperature,drink_type,product_profit
0,Espresso,3.00,0.80,Drink,Hot,Coffee,2.20
1,Americano,3.50,1.00,Drink,Hot,Coffee,2.50
2,Latte,4.50,1.20,Drink,Hot,Coffee,3.30
3,Cappuccino,4.25,1.15,Drink,Hot,Coffee,3.10
4,Flat White,4.75,1.25,Drink,Hot,Coffee,3.50
5,Mocha,4.80,1.40,Drink,Hot,Coffee,3.40
6,Iced Coffee,4.00,1.10,Drink,Cold,Coffee,2.90
7,Cold Brew,4.75,1.35,Drink,Cold,Coffee,3.40
8,Iced Latte,4.60,1.25,Drink,Cold,Coffee,3.35
9,Iced Matcha Latte,5.25,1.60,Drink,Cold,Tea,3.65


In [4]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   item_name          17 non-null     object 
 1   price              17 non-null     float64
 2   production_cost    17 non-null     float64
 3   item_type          17 non-null     object 
 4   drink_temperature  14 non-null     object 
 5   drink_type         14 non-null     object 
 6   product_profit     17 non-null     float64
dtypes: float64(3), object(4)
memory usage: 1.1+ KB


In [5]:
# merge profit from items to sales dataset
sales = pd.merge(original_sales, items[['item_name', 'item_type', 'product_profit']], 
                 how='inner', on='item_name')
sales["date_time"] = sales['date'] + " " + sales['time']
sales["date_time"] = pd.to_datetime(sales["date_time"])
sales['cup_profit'] = np.where(sales['surcharge'] == True, 0.5, 0)
sales['profit'] = sales['product_profit'] + sales['cup_profit']
sales = sales[['date_time', 'item_name', 'item_type', 'transaction_type',
               'own_cup', 'surcharge', 'customer_id', 'profit']]
sales

,date_time,item_name,item_type,transaction_type,own_cup,surcharge,customer_id,profit
0,2022-01-01 07:03:30,Cold Brew,Drink,Takeout,True,False,26946,3.40
1,2022-01-01 07:30:58,Iced Coffee,Drink,Takeout,True,False,24356,2.90
2,2022-01-01 08:32:23,Iced Latte,Drink,Takeout,False,False,3760,3.35
3,2022-01-01 08:45:03,Latte,Drink,Dine-in,True,False,5900,3.30
4,2022-01-01 09:03:47,Latte,Drink,Takeout,False,False,11589,3.30
...,...,...,...,...,...,...,...,...
32600,2024-12-31 14:07:18,Green Tea,Drink,Dine-in,False,False,21936,2.70
32601,2024-12-31 14:24:08,Latte,Drink,Takeout,True,False,15914,3.30
32602,2024-12-31 14:36:31,Chai Latte,Drink,Dine-in,True,False,15227,3.40
32603,2024-12-31 14:57:19,Cappuccino,Drink,Takeout,True,False,21314,3.10


In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32605 entries, 0 to 32604
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_time         32605 non-null  datetime64[ns]
 1   item_name         32605 non-null  object        
 2   item_type         32605 non-null  object        
 3   transaction_type  32605 non-null  object        
 4   own_cup           30461 non-null  object        
 5   surcharge         32605 non-null  bool          
 6   customer_id       32605 non-null  int64         
 7   profit            32605 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 1.8+ MB


In [7]:
# identify NaN values
sales.isna().sum()

date_time              0
item_name              0
item_type              0
transaction_type       0
own_cup             2144
surcharge              0
customer_id            0
profit                 0
dtype: int64

# Q1. How did drink sales change after the surcharge and rebrand, compared to merch?
- Surcharge and rebrand occurred on Nov 1, 2023
- Ramped up merch advertising also begins on Nov 1, 2023

In [8]:
sales_by_month = sales.copy()
sales_by_month['month'] = sales_by_month['date_time'].dt.to_period('M')

In [9]:
profit_by_month = (
    sales_by_month
    .groupby('month', as_index=False)
    .agg(profit=('profit', 'sum'))
)
profit_by_month.head()

,month,profit
0,2022-01,2884.40
1,2022-02,2758.00
2,2022-03,3421.45
3,2022-04,3072.25
4,2022-05,3114.80


In [10]:
# Set the year to be highlighted to a separate value in a new column
profit_by_month['highlight_rebrand'] = False
profit_by_month.loc[profit_by_month['month'] == pd.Period("2023-11", freq="M"), 'highlight_rebrand'] = True
profit_by_month['month_ts'] = profit_by_month['month'].dt.to_timestamp()

total_profit_plot = alt.Chart(profit_by_month).mark_bar().encode(
    x=alt.X('month_ts:T', 
            axis=alt.Axis(format='%b %Y', 
                          tickCount="month", 
                          labelOverlap=False, 
                          labelAngle=-45,
                          title='Month')),
    y=alt.Y("profit", axis=alt.Axis(title='profit ($)')),
    color=alt.Color('highlight_rebrand:N', legend=alt.Legend(title='Rebrand Month'))
).properties(
    width=700
)
total_profit_plot

alt.Chart(...)

In [11]:
sales['rebranded?'] = sales['date_time'] >= pd.to_datetime("2023-11-01")
sales

,date_time,item_name,item_type,transaction_type,own_cup,surcharge,customer_id,profit,rebranded?
0,2022-01-01 07:03:30,Cold Brew,Drink,Takeout,True,False,26946,3.40,False
1,2022-01-01 07:30:58,Iced Coffee,Drink,Takeout,True,False,24356,2.90,False
2,2022-01-01 08:32:23,Iced Latte,Drink,Takeout,False,False,3760,3.35,False
3,2022-01-01 08:45:03,Latte,Drink,Dine-in,True,False,5900,3.30,False
4,2022-01-01 09:03:47,Latte,Drink,Takeout,False,False,11589,3.30,False
...,...,...,...,...,...,...,...,...,...
32600,2024-12-31 14:07:18,Green Tea,Drink,Dine-in,False,False,21936,2.70,True
32601,2024-12-31 14:24:08,Latte,Drink,Takeout,True,False,15914,3.30,True
32602,2024-12-31 14:36:31,Chai Latte,Drink,Dine-in,True,False,15227,3.40,True
32603,2024-12-31 14:57:19,Cappuccino,Drink,Takeout,True,False,21314,3.10,True


In [12]:
# merch profit
merch_profit = sales_by_month[sales_by_month['own_cup'].isna()]
merch_profit = merch_profit.groupby('month', as_index=False).agg(profit=('profit', 'sum'))

merch_profit.head()

,month,profit
0,2022-01,332.0
1,2022-02,363.0
2,2022-03,356.0
3,2022-04,379.0
4,2022-05,385.0


In [13]:
merch_profit['month_ts'] = merch_profit['month'].dt.to_timestamp()

merch_profit_plot = alt.Chart(merch_profit).mark_line(color='green').encode(
    x=alt.X('month_ts:T', axis=alt.Axis(format='%b %Y', tickCount="month", labelOverlap=False, labelAngle=-45)),
    y="profit"
).properties(
    width=700 
)
merch_profit_plot

alt.Chart(...)

In [14]:
# drinks profit
drinks_profit = sales_by_month[~sales_by_month['own_cup'].isna()]
drinks_profit = drinks_profit.groupby('month', as_index=False).agg(profit=('profit', 'sum'))

drinks_profit.head()

,month,profit
0,2022-01,2552.40
1,2022-02,2395.00
2,2022-03,3065.45
3,2022-04,2693.25
4,2022-05,2729.80


In [15]:
drinks_profit['month_ts'] = drinks_profit['month'].dt.to_timestamp()

drinks_profit_plot = alt.Chart(drinks_profit).mark_line(color='pink').encode(
    x=alt.X('month_ts:T', axis=alt.Axis(format='%b %Y', tickCount="month", labelOverlap=False, labelAngle=-45)),
    y="profit"
).properties(
    width=700 
)
drinks_profit_plot

alt.Chart(...)

In [16]:
# combine everything and add a title
final_plot = (total_profit_plot + merch_profit_plot + drinks_profit_plot).properties(
    title='Monthly profit: Total vs Merch vs Drinks',
    width=660,
    height=400
)

final_plot

alt.LayerChart(...)

- The bar chart in the background represents the overall profit, with the rebranding month highlighted in orange. 
- The green line represents the profit from selling merch.
- The pink line represents the profit from selling drinks.
What do we notice:
- Dip from Sept - Dec 2022 from the construction period
- Surge in merch sales after the rebranding and increased merch advertising. But it has a slow decreasing trend as time passes. By the end of Dec 2024 it has dropped back to almost before.
- There was a spike in total profit after the rebrand and new surcharge, overall drinks profit is pretty stable over time, but the merch profit has a downward trend.
- Overall, the rebrand improved cafe's financial performance.

Next step:
- Correlation between merch purchases and reusable cup usage
- Are merch buyers more loyal? More sustainable in behaviour? --> yes from Jessie's analysis
- Do certain customers buy merch first, then bring cups?

# Q2. Did the increased advertising for the merch increased reusable cup use?
Analyze the proportion of reusable cup use before and after the increased advertising for the merch. We saw from the previous analysis that the merch profit/profit increases --> **created a small new profit stream**. **But did that actually encourage reusable cup habits?** --> Do people that buy the reusable cups actually use it?! For the people who use the reusable cups, did they buy them from us?

Potential regression problem
- what are the chances that the customer will buy reusable cups from us?
- what are the changes the customer will buy more than 1 reusable cups from us?

In [17]:
sales_by_month

,date_time,item_name,item_type,transaction_type,own_cup,surcharge,customer_id,profit,month
0,2022-01-01 07:03:30,Cold Brew,Drink,Takeout,True,False,26946,3.40,2022-01
1,2022-01-01 07:30:58,Iced Coffee,Drink,Takeout,True,False,24356,2.90,2022-01
2,2022-01-01 08:32:23,Iced Latte,Drink,Takeout,False,False,3760,3.35,2022-01
3,2022-01-01 08:45:03,Latte,Drink,Dine-in,True,False,5900,3.30,2022-01
4,2022-01-01 09:03:47,Latte,Drink,Takeout,False,False,11589,3.30,2022-01
...,...,...,...,...,...,...,...,...,...
32600,2024-12-31 14:07:18,Green Tea,Drink,Dine-in,False,False,21936,2.70,2024-12
32601,2024-12-31 14:24:08,Latte,Drink,Takeout,True,False,15914,3.30,2024-12
32602,2024-12-31 14:36:31,Chai Latte,Drink,Dine-in,True,False,15227,3.40,2024-12
32603,2024-12-31 14:57:19,Cappuccino,Drink,Takeout,True,False,21314,3.10,2024-12


In [18]:
# drop the NaNs because they are not buying drinks
reusable_cup_proportion = (sales_by_month
    .dropna()
    .groupby('month', as_index=False)
    .agg(total=('own_cup', 'size'),
         reusable=('own_cup', lambda x: (x == True).sum())))
                           
reusable_cup_proportion['proportion'] = (reusable_cup_proportion['reusable'] / 
                                         reusable_cup_proportion['total'])
reusable_cup_proportion.head()

,month,total,reusable,proportion
0,2022-01,803,335,0.417186
1,2022-02,760,318,0.418421
2,2022-03,967,389,0.402275
3,2022-04,855,329,0.384795
4,2022-05,865,371,0.428902


In [19]:
reusable_cup_proportion['highlight_rebrand'] = False
reusable_cup_proportion.loc[reusable_cup_proportion['month'] == pd.Period("2023-11", freq="M"), 'highlight_rebrand'] = True
reusable_cup_proportion['month_ts'] = reusable_cup_proportion['month'].dt.to_timestamp()

reusable_cup_proportion_plot = alt.Chart(reusable_cup_proportion).mark_bar().encode(
    x=alt.X('month_ts:T', 
            axis=alt.Axis(format='%b %Y', 
                          tickCount="month", 
                          labelOverlap=False, 
                          labelAngle=-45,
                          title='Month')),
    y=alt.Y("proportion", axis=alt.Axis(title='profit ($)')),
    color=alt.Color('highlight_rebrand:N', legend=alt.Legend(title='Rebrand Month'))
).properties(
    width=700
)
reusable_cup_proportion_plot

alt.Chart(...)

In [20]:
# plain line
line_layer = alt.Chart(reusable_cup_proportion).mark_line().encode(
    x=alt.X(
        'month_ts:T',
        axis=alt.Axis(
            format='%b %Y',
            tickCount='month',
            labelOverlap=False,
            labelAngle=-45,
            title='Month'
        )
    ),
    y=alt.Y(
        'proportion:Q',
        axis=alt.Axis(title='Proportion of people using reusable cups')
    )
)

# single highlight point on the rebrand month
point_layer = (
    alt.Chart(reusable_cup_proportion)
    .transform_filter(alt.datum.highlight_rebrand == True)
    .mark_point(size=150, filled=True, color='orange')
    .encode(
        x='month_ts:T',
        y='proportion:Q'
    )
)

reusable_cup_proportion_plot = (line_layer + point_layer).properties(width=700)
reusable_cup_proportion_plot

alt.LayerChart(...)

In [21]:
# mean of proportion before and after
reusable_cup_proportion['rebranded?'] = reusable_cup_proportion['month'] >= pd.Period("2023-11", freq="M")

mean_before_after = (
    reusable_cup_proportion
    .groupby('rebranded?')['proportion']
    .mean()
    .reset_index()
)

mean_before_after

,rebranded?,proportion
0,False,0.399572
1,True,0.487056


In [22]:
# perform a two proportion z-test to see if the results are significant and CLT 
before = reusable_cup_proportion[reusable_cup_proportion['rebranded?'] == False]
after  = reusable_cup_proportion[reusable_cup_proportion['rebranded?'] == True]

# total drinks sold
total_before = before['total'].sum()
total_after = after['total'].sum()

# number of reusable cups used
reusable_before = before['reusable'].sum()
reusable_after = after['reusable'].sum()

total_before, reusable_before, total_after, reusable_after

(np.int64(17933), np.int64(7169), np.int64(12528), np.int64(6098))

In [29]:
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm

# proportions
p1 = reusable_before / total_before  # before
p2 = reusable_after / total_after    # after

# difference
diff = p2 - p1

# standard error
se = np.sqrt(
    (p1 * (1 - p1)) / total_before +
    (p2 * (1 - p2)) / total_after
)

# 95% CI
z = norm.ppf(0.975)  # 1.96
lower = diff - z * se
upper = diff + z * se

print(f"95% CI for the difference: ({lower:.4f}, {upper:.4f})")

95% CI for the difference: (0.0757, 0.0983)


In [30]:
z_stat, p_two_tailed = proportions_ztest(count, nobs)

# convert to one-tailed
if diff > 0:
    p_one_tailed = p_two_tailed / 2
else:
    p_one_tailed = 1 - (p_two_tailed / 2)

p_one_tailed

np.float64(1.351301764427296e-51)

In [31]:
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_value:.5f}")

alpha = 0.05
if p_value < alpha:
    print("Result: Significant increase in reusable cup usage after the rebrand 🎉")
else:
    print("Result: No statistically significant change in reusable cup usage 😕")

Z-statistic: -15.066
P-value: 0.00000
Result: Significant increase in reusable cup usage after the rebrand 🎉


**But did that actually encourage reusable cup habits?** Yes! With a p-value < 0.001, it shows that there is a significant increase in reusable cup usage after the rebrand!
- Do people who buy the reusable cups actually use them?
- For the people who use the reusable cups, did they buy them from us?